# Evaluation
Evaluate accuracy of vision model

 Finding the nea image names where i last stopped evaluating on

In [14]:
import json
data_cleaned = json.loads(open('./data.json').read())
item_names = [i['item'] for i in data_cleaned]
print(f'total items in json file: {len(item_names)}')
print(f'total unique items in json file: {len(set(item_names))}')


total items in json file: 322
total unique items in json file: 271


In [15]:
import pandas as pd

df = pd.read_csv('./manual_evaluation_with_similar_items.csv')
df.head()



,filename,ground_truth,predictions,similarity,evaluation,manual evaluation,remarks
0,../images/test/batch1/Bedsheet (2).jpg,Bedsheet,['Bedsheet'],1.0,1.0,1.0,NaN
1,../images/test/batch1/Bedsheet (3).jpg,Bedsheet,['Bedsheet'],1.0,1.0,1.0,NaN
2,../images/test/batch1/Bedsheet (4).jpg,Bedsheet,['Bedsheet'],1.0,1.0,1.0,NaN
3,../images/test/batch1/Bedsheet (5).jpg,Bedsheet,['Bedsheet'],1.0,1.0,1.0,NaN
4,../images/test/batch1/Bedsheet (6).jpg,Bedsheet,['Bedsheet'],1.0,1.0,1.0,NaN


In [16]:
evaluated_names = df['ground_truth'].unique()
print(len(evaluated_names))



99


In [17]:
names_to_evaluate = set(item_names).difference(set(evaluated_names))
len(names_to_evaluate)



183

In [18]:
99/271

0.36531365313653136

In [19]:
# output to csv
import csv
with open('names_to_evaluate.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerow(['name'])
    writer.writerows([[name] for name in names_to_evaluate])



---

In [82]:
l = [
        "Paper",
        "Paper bag",
        "Paper box",
        "Paper cup",
        "Paper disposables",
        "Printed paper (Glossy and non-glossy)"
    ]
l

['Paper',
 'Paper bag',
 'Paper box',
 'Paper cup',
 'Paper disposables',
 'Printed paper (Glossy and non-glossy)']

# Finding the unique item names in data json file

In [2]:
# read from data.json
import json
items = []
with open('data.json', 'r') as file:
    items = json.load(file)

print(len(items))
items[:5]



322


[{'material': 'Paper',
  'item': 'Printed paper (Glossy and non-glossy)',
  'recyclable': True,
  'instructions': 'Make sure it is clean before recycling.',
  'similar_items': ['Printed paper (Glossy and non-glossy)']},
 {'material': 'Paper',
  'item': 'Writing paper',
  'recyclable': True,
  'instructions': 'Make sure it is clean before recycling.',
  'similar_items': ['Writing paper']},
 {'material': 'Paper',
  'item': 'Paper',
  'recyclable': True,
  'instructions': 'Make sure it is clean before recycling.',
  'similar_items': ['Paper',
   'Exam Papers',
   'Notes',
   'Notebook',
   'Mail',
   'Letter',
   'Bill',
   'Foolscap Paper',
   'Exercise Book']},
 {'material': 'Others',
  'item': 'Clothes',
  'recyclable': False,
  'instructions': "Clothes should be donated if they are in good condition. <br/><br/> Click <a href='https://www.nea.gov.sg/our-services/waste-management/donation-resale-and-repair-channels/' target='_blank' style='color:black; font-weight:600; text-decoration: 

In [4]:
unique_items = set([item['item'] for item in items])
len(unique_items)



271

In [5]:
item_count = {}
for item in items:
    if item['item'] in item_count:
        item_count[item['item']] += 1
    else:
        item_count[item['item']] = 1

# print the items with count > 1
for item, count in item_count.items():
    if count > 1:
        print(item, count)



Printed paper (Glossy and non-glossy) 2
Writing paper 2
Flyer (Glossy and non-glossy) 2
Magazine (Glossy and non-glossy) 2
Telephone directory 2
Envelope (With and without plastic window) 2
Plastic envelope 2
Red packet 2
Namecard 2
Greeting card 2
Shredded paper 2
Carton box 2
Cardboard box 2
Printed paper box 2
Plastic bottle 2
Plastic container 2
Soft drink bottle 2
Carbonated drink bottle 2
Medicine bottle 2
Fruit box 2
Ziplock bag 2
Plastic packaging 2
Plastic disposables 2
Plastic crockery 2
Plastic packaging with foil 2
Potato chip bags 2
Expired credit cards 2
Plastic packaging contaminated with food/oil stains 2
Glass bottle 2
Wine bottle 2
Liquor bottle 2
Food glass bottle 2
Sauce bottle 2
Condiment bottle 2
Jam spread bottle 2
Food jars 2
Cosmetic glass bottle 2
Perfume glass bottle 2
Drinking glass 2
Paint container 2
Paint cans 2
Laptop 3
Household battery 2
Food waste 2
Stationery 2
Furniture 2
Sports shoes 2
School shoes 2
Tablet computer 2
Mobile phone 2


# Putting similar items into item names

In [6]:
# read from data.json
import json
items = []
with open('data.json', 'r') as file:
    items = json.load(file)

print(len(items))




322


In [7]:
all_items = list(set([item['item'] for item in items]))
len(all_items)


271

In [9]:
for item in items:
    all_items.extend(item['similar_items'])
print(len(all_items))
all_items[:5]



['Medicine glass bottle',
 'Computer battery',
 'Power-assisted bicycle',
 'Calendar',
 'Umbrella']

In [13]:
# clean to trim any leading and trailing space, remove any duplicates, sort a to z
all_items = list(set([item.strip() for item in all_items]))
all_items = sorted(all_items)
print(len(all_items))
all_items[:5]



596


['', '100 Plus', '100 plus', '9-Volt battery', 'ADAT']

In [14]:
all_items = all_items[1:]
len(all_items)



595

In [17]:
# output to csv
import csv
with open('all_items.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerow(['item'])
    writer.writerows([[item] for item in all_items])



In [20]:
#output to json
with open('all_items.json', 'w') as file:
    json.dump(all_items, file)



---

In [20]:
from langchain_openai import ChatOpenAI

vision_model = ChatOpenAI(model="gpt-4o", temperature=0)


In [1]:
# base64 encode an image
import base64

def encode_image(image_path):
    """Getting the base64 string"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


img_base64 = encode_image(
    "../images/rubc_cube.jpg"
)
# Write the base64 string to a text file
with open("encoded_image.txt", "w") as file:
    file.write(img_base64)


In [22]:
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List
from enum import Enum

class Items(BaseModel):
    items: List[str] = Field(description="List of item names")


In [53]:
# excluding similar items
NEA_ITEM_NAMES = [
    "9-Volt battery",
    "Aerosol cans",
    "Air Conditioner",
    "Alkaline battery",
    "Audio sound system",
    "Bag",
    "Glass Bakeware",
    "Battery charger",
    "Bedsheet",
    "Glass beer bottle",
    "Beer can",
    "Beverage carton",
    "Beverage glass bottle",
    "Beverage metal can",
    "Bio-degradable bag",
    "Biscuit tin",
    "Blanket",
    "Blister pack",
    "Plastic bodywash bottle",
    "Books",
    "Borosilicate glassware",
    "Bread bag",
    "Briefcase",
    "Brochure (Glossy and non-glossy)",
    "Bubble tea cups",
    "Bubble wrap",
    "Bulky waste",
    "Button cell battery",
    "CD",
    "CD casing",
    "CD player",
    "Calendar",
    "Canned food",
    "Carbonated drink bottle",
    "Carbonated drink can",
    "Cardboard box",
    "Carton box",
    "Cassette",
    "Ceramic plate",
    "Ceramic products",
    "Chairs",
    "Child seat",
    "Cigarettes",
    "Clean aluminium foil",
    "Clean aluminium tray",
    "Cling film",
    "Clothes",
    "Coffee capsules",
    "Computer battery",
    "Computer mouse",
    "Glass condiment bottle",
    "Cooking pot",
    "Cosmetic glass bottle",
    "Cotton bud",
    "Cotton wool",
    "Crayon drawing",
    "Crystal glass",
    "Curtains",
    "DVD",
    "DVD casing",
    "DVD player",
    "Desktop computer",
    "Desktop monitor",
    "Detergent bottle",
    "Diaper",
    "Dirty aluminium foil",
    "Dirty aluminium tray",
    "Disposable shaver",
    "Disposable wooden chopsticks",
    "Docking station",
    "Drink packet",
    "Drinking glass",
    "Drinking straw",
    "Hairdryer",
    "E-scooter",
    "Electric bicycle",
    "Electric fan",
    "Electric kettle",
    "Electric mobility devices",
    "Electric mobility scooter",
    "Electric scooter",
    "Electric shaver",
    "Electronic cables",
    "Electronic waste",
    "Envelope (With and without plastic window)",
    "Expired credit cards",
    "Facial cleanser bottle",
    "Fluorescent bulb",
    "Fluorescent lamp",
    "Flyer (Glossy and non-glossy)",
    "Food blender",
    "Food glass bottle",
    "Food jars",
    "Food metal can",
    "Food processor",
    "Food tin",
    "Food waste",
    "Football shoes (without metal studs)",
    "Fridge",
    "Fruit box",
    "Furniture",
    "Gaming console",
    "Gift wrapping paper",
    "Glad wrap",
    "Glass bottle",
    "Glass cup",
    "Glass plate",
    "Glass with metal wires",
    "Glassware",
    "Glitter paper",
    "Greeting card",
    "Hard disk drive",
    "Horticultural waste",
    "Household battery",
    "Incandescent bulb",
    "Incandescent lamp",
    "Jam spread bottle",
    "Joss sticks",
    "Juice packet",
    "Keyboard",
    "LED bulb",
    "LED lamp",
    "Lamp",
    "Lamp fixture",
    "Lamp stand",
    "Laptop",
    "Large household appliances",
    "Leftover medicine",
    "Light bulb",
    "Liquor bottle",
    "Luggage bag",
    "Magazine (Glossy and non-glossy)",
    "Magazine wrapper",
    "Masks",
    "Medals",
    "Medicine bottle",
    "Medicine glass bottle",
    "Melamine cups",
    "Melamine plates",
    "Melamine products",
    "Metal accessories",
    "Metal bottle cap",
    "Metal container",
    "Metal cutlery",
    "Microwave oven",
    "Milk bottles",
    "Milk carton",
    "Mineral water bottle",
    "Mirror",
    "Mobile phone",
    "Mobile phone battery",
    "Modem",
    "Mouthwash bottle",
    "Music player",
    "Namecard",
    "Newsletter",
    "Newspaper",
    "Non-food metal container",
    "Oven-safe food containers",
    "Oxo-degradable bag",
    "Paint cans",
    "Metal paint container",
    "Paper",
    "Paper Packaging (printed paper box etc)",
    "Paper bag",
    "Paper box",
    "Paper cup",
    "Paper disposables",
    "Paper egg trays",
    "Paper packaging contaminated with food",
    "Paper packaging with food",
    "Paper plate",
    "Paper receipt",
    "Paper towel",
    "Paper towel tube",
    "Pen",
    "Pencil",
    "Perfume glass bottle",
    "Personal mobility devices",
    "Pizza boxes",
    "Plant waste",
    "Plastic bag",
    "Plastic bottle",
    "Plastic bottle cap",
    "Plastic clothes hanger",
    "Plastic container",
    "Plastic crockery",
    "Plastic cups",
    "Plastic cutlery",
    "Plastic disposables",
    "Plastic egg trays",
    "Plastic envelope",
    "Plastic film",
    "Plastic food wrap",
    "Plastic packaging",
    "Plastic packaging contaminated with food/oil stains",
    "Plastic packaging for packet drink",
    "Plastic packaging with foil",
    "Plastic takeaway food container",
    "Polystyrene foam product",
    "Porcelain",
    "Portable charger",
    "Potato chip bags",
    "Power bank",
    "Power-assisted bicycle",
    "Printed paper (Glossy and non-glossy)",
    "Printed paper box",
    "Printer",
    "Pyrex glassware",
    "Radio",
    "Rechargeable battery",
    "Red packet",
    "Refrigerator",
    "Ribbons",
    "Rice cooker",
    "Router",
    "Rusty metal cans",
    "Sanitary pad",
    "Saran wrap",
    "Sauce bottle",
    "School shoes",
    "Shampoo",
    "Shampoo bottle",
    "Shoes",
    "Shredded paper",
    "Soap bottle",
    "Soft drink bottle",
    "Soft drink can",
    "Speaker",
    "Spectacles",
    "Sports shoes",
    "Spray cans",
    "Standing fan",
    "Stationery",
    "Steel wool",
    "Styrofoam",
    "Styrofoam clamshell container",
    "Styrofoam cup",
    "Supplement glass bottle",
    "TV",
    "Table",
    "Tablet computer",
    "Tea pot",
    "Telephone directory",
    "Television",
    "Tempered glass",
    "Textbooks",
    "Textile",
    "Tissue box",
    "Tissue box packaging",
    "Tissue paper",
    "Toaster oven",
    "Toilet paper",
    "Toilet paper packaging",
    "Toilet roll tube",
    "Toys",
    "Glass tube",
    "Umbrella",
    "Vacuum cleaner",
    "Vase",
    "Video tape",
    "Washing machine",
    "Water bottle",
    "Wax paper",
    "Wet wipes",
    "Windows",
    "Wine bottle",
    "Wine glass",
    "Wooden chopsticks",
    "Writing paper",
    "Ziplock bag",
]



In [23]:
# including similar items
NEA_ITEM_NAMES = [
    "100 Plus", "100 plus", "9-Volt battery", "ADAT", "ASUS", "ATM Card", "Absolut", "Acer", "Adidas", "Aerosol cans", "Air Conditioner", "Alkaline battery", "Almon Buuter", "Anchor", "Android", "Ang Pow", "Anniversary Card", "Apple", "Apple Juice", "Aqua", "Asahi", "Asics", "Assessment Books", "Audio sound system", "Bag", "Bakeware", "Baking Paper", "Bank Card", "Battery charger", "Bed Frame", "Bedsheet", "Beer bottle", "Beer can", "Belvedere", "BenjaminmooreYeo's", "Berocca", "Beverage carton", "Beverage glass bottle", "Beverage metal can", "Bill", "Bio-degradable bag", "Birthday Card", "Biscuit tin", "Blackmores", "Blanket", "Blister pack", "Blouse", "Bodywash bottle", "Bombay Sapphire", "Bonjour", "Books", "Borosilicate glassware", "Bowl", "Bread bag", "Breda", "Briefcase", "Brochure (Glossy and non-glossy)", "Bubble tea cups", "Bubble wrap", "Bulky waste", "Business Card", "Button cell battery", "CD", "CD Casing", "CD casing", "CD player", "Calbee", "Calendar", "Calsberg", "Camisole", "Canned food", "Carbonated drink bottle", "Carbonated drink can", "Cardboard box", "Carton box", "Cassette", "Cellphone", "Ceramic plate", "Ceramic products", "Cereal box", "Chair", "Chairs", "Champagne", "Child seat", "Chilli", "Chilli Sauce Bottle", "Chivas", "Christmas Card", "Cigarettes", "Clean aluminium foil", "Clean aluminium tray", "Cling film", "Clothes", "Coffee Table", "Coffee capsules", "Coke", "Comic books", "Compact Disc", "Computer battery", "Computer mouse", "Condiment bottle", "Containers", "Cooking pot", "Corona", "Correction liquid", "Correction tape", "Cosmetic glass bottle", "Cotton Bud", "Cotton Wool", "Cotton bud", "Cotton wool", "Couch", "Crayon drawing", "Crocs", "Crystal glass", "Curtains", "DVD", "DVD casing", "DVD player", "Dasani", "Debit Card", "Dell", "Desktop computer", "Desktop monitor", "Detergent bottle", "Diaper", "Diary", "Dirty aluminium foil", "Dirty aluminium tray", "Disposable Knife", "Disposable bowl", "Disposable cup", "Disposable cutlery", "Disposable fork", "Disposable plate", "Disposable shaver", "Disposable spoon", "Disposable wooden chopsticks", "Docking station", "Dress shoes", "Drink packet", "Drinking glass", "Drinking straw", "Dryer", "Duracell", "E-scooter", "Electric bicycle", "Electric fan", "Electric kettle", "Electric mobility devices", "Electric mobility scooter", "Electric scooter", "Electric shaver", "Electronic cables", "Electronic waste", "Energizer", "Enriched", "Envelope (With and without plastic window)", "Eveready", "Evian", "Exam Papers", "Exercise Book", "Exercise book", "Expired credit cards", "F&N", "FNN", "Fabric", "Facial cleanser bottle", "Fan", "Fanta", "Farm Fresh", "Fiji", "Fire-Wire", "Fluorescent bulb", "Fluorescent lamp", "Flyer (Glossy and non-glossy)", "Food blender", "Food glass bottle", "Food jars", "Food metal can", "Food processor", "Food tin", "Food waste", "Foolscap Paper", "Football shoes (without metal studs)", "Fork", "Fridge", "Fruit Container", "Fruit Juice Packet", "Fruit box", "Furniture", "Gaming console", "Gardenia", "Gift", "Gift bag", "Gift wrapping paper", "Gifts", "Gin", "Glad wrap", "Glass Bakeware", "Glass Tube", "Glass bottle", "Glass cup", "Glass plate", "Glass with metal wires", "Glasses", "Glassware", "Glitter paper", "Google Phone", "Graph paper", "Green Tea", "Green packet", "Greenfield", "Greeting card", "Grey Goose", "Guinness", "HDMI Cable", "HL", "HP", "Hammerite", "Handphone", "Hard disk drive", "Heaven & Earth", "Heels", "Heineken", "Hendricks", "Highlighter", "Hoegarden", "Hojicha Tea", "Honey", "Hong Bao", "Horticultural waste", "Household battery", "IMac", "IPad", "Ice Lemon Tea", "Ice Mountain", "Incandescent bulb", "Incandescent lamp", "Ipad", "Iphone", "Jack Daniels", "Jacket", "Jagermeister", "Jam spread bottle", "Johnny Walker", "Joss sticks", "Jotter book", "Jotun", "Journal", "Juice", "Juice packet", "Ketchup", "Ketchup Bottle", "Keyboard", "Kingfisher", "Kitchen Roll", "Kitchen Towel", "Knife", "Kronenbourgh", "LED bulb", "LED lamp", "Lamp", "Lamp fixture", "Lamp stand", "Laptop", "Large household appliances", "Lay's", "Leaflet", "Leftover medicine", "Lenovo", "Letter", "Light bulb", "Lightning cable", "Liquor bottle", "Luggage bag", "Lychee Tea", "Macbook", "Macbook Air", "Macbook Pro", "Magazine (Glossy and non-glossy)", "Magazine wrapper", "Mail", "Make up", "Marigold", "Masks", "Mattress", "Mayonaise", "Meadows", "Medals", "Medicine Pack", "Medicine bottle", "Medicine glass bottle", "Meiji", "Melamine cups", "Melamine plates", "Melamine products", "Metal accessories", "Metal bottle cap", "Metal container", "Metal cutlery", "Microwave oven", "Milk bottles", "Milk carton", "Milo", "Mineral water bottle", "Mirror", "Mobile phone", "Mobile phone battery", "Modem", "Monkey Shoulder", "Mouth wash", "Mouthwash bottle", "Music player", "Mustard", "Namecard", "Nescafe", "Nespresso", "New Balance", "Newsletter", "Newspaper", "Nike", "Nippon Paint", "Nokia", "Non-food metal container", "Notebook", "Notes", "Notice", "Novel", "Oatly", "Oatside", "Oolong Tea", "Orange Juice", "Oven-safe food containers", "Oxo-degradable bag", "Packaging", "Paint cans", "Paint container", "Pampers", "Pamphlet", "Pants", "Paper", "Paper Packaging (printed paper box etc)", "Paper bag", "Paper box", "Paper cup", "Paper disposables", "Paper egg trays", "Paper packaging contaminated with food", "Paper packaging with food", "Paper plate", "Paper receipt", "Paper towel", "Paper towel tube", "Parchment Paper", "Peanut Butter", "Peel Fresh", "Pen", "Pencil", "Pepsi", "Perfume glass bottle", "Personal mobility devices", "Pill Bottle", "Pizza Box", "Pizza boxes", "Plant waste", "Plaster", "Plastic Cup Lids", "Plastic Fork", "Plastic Knife", "Plastic bag", "Plastic bottle", "Plastic bottle cap", "Plastic bowl", "Plastic clothes hanger", "Plastic container", "Plastic crockery", "Plastic cups", "Plastic cutlery", "Plastic disposables", "Plastic egg trays", "Plastic envelope", "Plastic film", "Plastic food wrap", "Plastic packaging", "Plastic packaging contaminated with food/oil stains", "Plastic packaging for packet drink", "Plastic packaging with foil", "Plastic plate", "Plastic takeaway food container", "Plate", "Pokka", "Polystyrene foam product", "Porcelain", "Portable charger", "Post-it", "Potato chip bags", "Power bank", "Power-assisted bicycle", "Present", "Presents", "Printed paper (Glossy and non-glossy)", "Printed paper box", "Printer", "Prosecco", "Publication", "Puma", "Pumps", "Pyrex glassware", "Radio", "Rafflespaint", "Razer", "Razor", "Rechargeable battery", "Red Wine", "Red packet", "Redoxon", "Reebok", "Refrigerator", "Ribbons", "Ribena", "Rice cooker", "Roku Gin", "Ronseal", "Router", "Ruffles", "Ruler", "Rum", "Rusty metal cans", "Samsung", "Sandles", "Sanitary pad", "Saran wrap", "Sauce bottle", "Saucer", "School Diary", "School shoes", "Scotts", "Seal bag", "Seasons", "Serving Bowl", "Serving Plate", "Sesame oil", "Shampoo", "Shampoo bottle", "Shirts", "Shoes", "Shorts", "Shredded paper", "Singlet", "Sketchers", "Skirt", "Smartwater", "Smirnoff", "Sneakers", "Soap bottle", "Soda cans", "Sofa", "Soft drink bottle", "Soft drink can", "Soy Sauce", "Speaker", "Spectacles", "Spoon", "Spork", "Sports shoes", "Spray cans", "Spread", "Sprite", "Stamps", "Standing fan", "Stationery", "Steel wool", "Storybook", "Styrofoam", "Styrofoam clamshell container", "Styrofoam cup", "Sunkist", "Sunshine", "Super Value", "Supplement glass bottle", "T-Shirt", "TRS", "TS", "TV", "Table", "Tablet computer", "Takeaway", "Tanquery", "Tea pot", "Telephone directory", "Television", "Tempered glass", "Ten Year Series", "Tequila", "Textbooks", "Textile", "The Botanist", "Tie", "Tiger", "Tiger Brand", "Tissue box", "Tissue box packaging", "Tissue paper", "Toaster oven", "Toilet paper", "Toilet paper packaging", "Toilet roll tube", "Toiletries", "Top One", "Torres", "Toys", "Tube", "Tupperware", "Twisties", "Type B", "Type C", "Tyrrell's", "USB", "USB C", "Umbrella", "Under Armour", "Vacuum cleaner", "Vase", "Video tape", "Vitasoy", "Volvic", "Washing machine", "Water bottle", "Wax paper", "Wet Wipes", "Wet wipes", "Whisky", "White Wine", "Windows", "Wine bottle", "Wine glass", "Wooden chopsticks", "Wrapper", "Writing paper", "XLR", "Yeo's", "Ziplock bag", "basketball", "biscuits", "boots", "cabinet", "candy", "chocolate", "cookies", "crackers", "cupboard", "cushion", "dining table", "drumstick", "eraser", "fish", "food peels", "football", "fragrance", "golf ball", "gummy bears", "hua tiao jiu", "iron", "ironing board", "leftovers", "marker", "mason jar", "meat", "nintendo", "noodles", "pH Balancer", "permanent Marker", "pizza", "plastic bag", "plastic bubble tea carrier", "playstation", "rice", "robot cleaner", "robot vacuum", "roller blades", "roller skates", "rotten food", "seafood", "shoe rack", "shot glass", "spoiled food", "stool", "sweets", "switch", "tennis ball", "vegetables", "whiteboard marker", "whskey glass", "wii"]

In [24]:
len(NEA_ITEM_NAMES)

595

In [25]:
import os
IMAGE_DIR = '../images/test/batch3'
filenames = [file for file in os.listdir(IMAGE_DIR) if file.endswith('jpeg') or file.endswith('jpg') or file.endswith('webp')]
filenames = [os.path.join(IMAGE_DIR, filename) for filename in filenames]
print(f"Number of images: {len(filenames)}")
filenames[:5]


Number of images: 154


['../images/test/batch3/Newsletter (4).jpeg',
 '../images/test/batch3/Textbooks (3).jpg',
 '../images/test/batch3/Disposable spoon (3).jpg',
 '../images/test/batch3/Computer mouse (2).jpg',
 '../images/test/batch3/bodywash bottle (3).jpeg']

In [26]:
from math import ceil
from PIL import Image

def calculate_openai_image_tokens(image_filepath: str):
    image = Image.open(image_filepath)
    width, height = image.size

    if width > 2048 or height > 2048:
        aspect_ratio = width / height
        if aspect_ratio > 1:
            width, height = 2048, int(2048 / aspect_ratio)
        else:
            width, height = int(2048 * aspect_ratio), 2048
            
    if width >= height and height > 768:
        width, height = int((768 / height) * width), 768
    elif height > width and width > 768:
        width, height = 768, int((768 / width) * height)

    tiles_width = ceil(width / 512)
    tiles_height = ceil(height / 512)
    total_tokens = 85 + 170 * (tiles_width * tiles_height)
    
    return total_tokens



In [27]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens


In [28]:
from langchain_core.prompts import ChatPromptTemplate

image_prompt = """
I have an image containing items that I am unsure of whether they are recyclable. Please help me to identify the item(s) in the image.
Return the best or closest matching item(s) you have identified to the following NEA_ITEM_NAMES: {NEA_ITEM_NAMES}

If there are best or closest matching item(s), you must return the item name according to the NEA_ITEM_NAMES.
Return the item as "Other" if the item is not in the list of NEA_ITEM_NAMES.

Return the answer as JSON output according to the following schema:
{{
    "items": ['item1', 'item2', ...]
}}

"""

image_prompt = image_prompt.format(NEA_ITEM_NAMES=NEA_ITEM_NAMES)

system_message = "You are an expert on answering questions briefly and accurately about recycling in Singapore. Your name is Bloo. Users may send you images of items to check if the items can be recycled, and your task is to correctly identify what are the items in the image."

image_prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system_message,
        ),
        (
            "human",
            [
                {"type": "text", "text": "{image_prompt}"},
                {
                    "type": "image_url",
                    "image_url": {"url": "data:image/jpeg;base64,{img_base64}"},
                },
            ],
        ),
    ]
)

vision_model_json_output = vision_model.with_structured_output(schema=Items)

chain_text_stream = image_prompt_template | vision_model_json_output


In [29]:
encoding = tiktoken.encoding_name_for_model('gpt-4o')

image_prompt_tokens_count = num_tokens_from_string(image_prompt, encoding)
system_message_tokens_count = num_tokens_from_string(system_message, encoding)
print(f"""
      System message token count: {system_message_tokens_count}
      Image prompt token count: {image_prompt_tokens_count}
      Total prompt token count: {image_prompt_tokens_count + system_message_tokens_count}""")



      System message token count: 52
      Image prompt token count: 2780
      Total prompt token count: 2832


In [30]:
# imports
import random
import time
import openai

# define a retry decorator
def retry_after_delay(
    func,
    errors: tuple = (openai.RateLimitError,),
):

    def wrapper(*args, delay: float = 65, max_retries: int = 3, **kwargs):
        # Initialize variables
        num_retries = 0

        # Loop until a successful response or max_retries is hit or an exception is raised
        while True:
            try:
                return func(*args, **kwargs)

            # Retry on specified errors
            except errors as e:
                print(e, end='\n\n')
                # Increment retries
                num_retries += 1

                # Check if max retries has been reached
                if num_retries > max_retries:
                    raise Exception(
                        f"Maximum number of retries ({max_retries}) exceeded."
                    )

                # Sleep for the delay
                time.sleep(delay)

            # Raise exceptions for any errors not specified
            except Exception as e:
                raise e

    return wrapper


@retry_after_delay
def batch_image_prediction_with_retry(filenames, **kwargs):
    batch_res = chain_text_stream.batch([{'image_prompt': image_prompt, 'img_base64': encode_image(filename)} for filename in filenames]) # batch_res is List[Items]

    return batch_res


In [31]:
# Checkpoint save to csv
import csv
def save_to_csv(csv_filename: str, filenames: List[str], res: List[Items]):
    file_exists = os.path.exists(csv_filename)
    new_rows = [(filename, res.items) for filename, res in zip(filenames, res)]
    with open(csv_filename, 'a', newline='') as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(['filename', 'predictions'])
        writer.writerows(new_rows)

    print(f'Saved {len(filenames)} rows to {csv_filename}')


In [32]:
# Batch invoke the model, retrying after 65 seconds on rate limit error

start_index = 0
end_index = 0
curr_tokens = 0

# obtained from openai project limits based on project tier
TPM = 30_000

res = []

results_csv_filename = 'results_batch3_with_similar_items.csv'

while end_index < len(filenames):
    image_path = filenames[end_index]
    image_tokens_count = calculate_openai_image_tokens(image_path)
    curr_tokens += system_message_tokens_count + image_prompt_tokens_count + image_tokens_count + 1024 # last constant at the end is buffer for input and output formatting + the response from the model

    if curr_tokens < TPM:
        end_index += 1

    else:
        print(f'Exceeding {TPM} TPM. Current tokens {curr_tokens}')
        img_base64 = encode_image(image_path)
        filenames_to_invoke = filenames[start_index: end_index]

        print("batch invoking for filenames:\n", ',\n'.join(filenames_to_invoke), end='\n----------------------------\n')

        batch_res = batch_image_prediction_with_retry(filenames_to_invoke) # batch_res is List[Items]
        save_to_csv(results_csv_filename, filenames_to_invoke, batch_res)

        start_index = end_index
        res.extend(batch_res)

        curr_tokens = 0 # Rest token count for next batch

    if end_index == len(filenames):
        print('Last file reached.')
        filenames_to_invoke = filenames[start_index: end_index]
        print("batch invoking for filenames:\n", ',\n'.join(filenames_to_invoke), end='\n----------------------------\n')

        batch_res = batch_image_prediction_with_retry(filenames_to_invoke) # batch_res is List[Items]
        save_to_csv(results_csv_filename, filenames_to_invoke, batch_res)
        res.extend(batch_res)

res



Exceeding 30000 TPM. Current tokens 32687
batch invoking for filenames:
 ../images/test/batch3/Newsletter (4).jpeg,
../images/test/batch3/Textbooks (3).jpg,
../images/test/batch3/Disposable spoon (3).jpg,
../images/test/batch3/Computer mouse (2).jpg,
../images/test/batch3/bodywash bottle (3).jpeg,
../images/test/batch3/Ribbons (2).jpg
----------------------------
Saved 6 rows to results_batch3_with_similar_items.csv
Exceeding 30000 TPM. Current tokens 32347
batch invoking for filenames:
 ../images/test/batch3/food blender (4).jpg,
../images/test/batch3/Personal mobility devices.jpg,
../images/test/batch3/blister pack (2).webp,
../images/test/batch3/Vacuum cleaner.jpg,
../images/test/batch3/Toaster oven.jpg,
../images/test/batch3/Notebook (3).jpeg
----------------------------
Saved 6 rows to results_batch3_with_similar_items.csv
Exceeding 30000 TPM. Current tokens 32517
batch invoking for filenames:
 ../images/test/batch3/Glass Tube.jpg,
../images/test/batch3/Sanitary pad (4).jpg,
../im

[Items(items=['Newsletter']),
 Items(items=['Books']),
 Items(items=['Disposable spoon']),
 Items(items=['Computer mouse']),
 Items(items=['Soap bottle', 'Shampoo bottle', 'Body lotion bottle']),
 Items(items=['Bedsheet']),
 Items(items=['Food blender']),
 Items(items=['Electric mobility scooter']),
 Items(items=['Blister pack']),
 Items(items=['robot cleaner']),
 Items(items=['Toaster oven']),
 Items(items=['Notebook']),
 Items(items=['Glass Tube']),
 Items(items=['Sanitary pad', 'Tampons']),
 Items(items=['Lamp', 'Lamp fixture', 'Lamp stand']),
 Items(items=['Iphone']),
 Items(items=['Shoes']),
 Items(items=['Rice cooker']),
 Items(items=['Disposable shaver']),
 Items(items=['Disposable spoon']),
 Items(items=['Audio sound system']),
 Items(items=['Gaming console']),
 Items(items=['Toaster oven']),
 Items(items=['Wooden chopsticks']),
 Items(items=['Steel wool']),
 Items(items=['Disposable shaver']),
 Items(items=['Audio sound system']),
 Items(items=['Battery charger', 'Electronic c

## Read results into dataframe and save to json
Read csv results into dataframe, extract the ground truth from the filename, save to json to pass as input to LLM for similarity evaluation


In [33]:
# put results into dataframe
import pandas as pd
df = pd.read_csv('results_batch3_with_similar_items.csv')
print(df.shape)
df.head()



(154, 2)


,filename,predictions
0,../images/test/batch3/Newsletter (4).jpeg,['Newsletter']
1,../images/test/batch3/Textbooks (3).jpg,['Books']
2,../images/test/batch3/Disposable spoon (3).jpg,['Disposable spoon']
3,../images/test/batch3/Computer mouse (2).jpg,['Computer mouse']
4,../images/test/batch3/bodywash bottle (3).jpeg,"['Soap bottle', 'Shampoo bottle', 'Body lotion..."


Evaluation
- Extract the ground truth from the filename
- Save to json
- Pass to LLM for evaluation
- Save results into dataframe
- Export dataframe to csv


In [34]:
import os

df['ground_truth'] = df['filename'].apply(lambda x: os.path.splitext(os.path.basename(x))[0])
df.head()

,filename,predictions,ground_truth
0,../images/test/batch3/Newsletter (4).jpeg,['Newsletter'],Newsletter (4)
1,../images/test/batch3/Textbooks (3).jpg,['Books'],Textbooks (3)
2,../images/test/batch3/Disposable spoon (3).jpg,['Disposable spoon'],Disposable spoon (3)
3,../images/test/batch3/Computer mouse (2).jpg,['Computer mouse'],Computer mouse (2)
4,../images/test/batch3/bodywash bottle (3).jpeg,"['Soap bottle', 'Shampoo bottle', 'Body lotion...",bodywash bottle (3)


In [35]:
import re

def clean_text(text):
    # This regex removes a trailing space followed by parentheses containing only digits
    return re.sub(r'\s+\(\d+\)$', '', text)

# Example usage:
df['ground_truth'] = df['ground_truth'].apply(clean_text)
df.head()

,filename,predictions,ground_truth
0,../images/test/batch3/Newsletter (4).jpeg,['Newsletter'],Newsletter
1,../images/test/batch3/Textbooks (3).jpg,['Books'],Textbooks
2,../images/test/batch3/Disposable spoon (3).jpg,['Disposable spoon'],Disposable spoon
3,../images/test/batch3/Computer mouse (2).jpg,['Computer mouse'],Computer mouse
4,../images/test/batch3/bodywash bottle (3).jpeg,"['Soap bottle', 'Shampoo bottle', 'Body lotion...",bodywash bottle


In [36]:
df[['ground_truth', 'predictions']].to_json('results_batch3_with_similar_items.json', orient='records')



In [37]:
import json
# Open the JSON file and read its contents into a string
with open('results_batch3_with_similar_items.json', 'r') as file:
    result_list = json.load(file)

# Now result_list contains the contents of the JSON file as a string
result_list[:5]

[{'ground_truth': 'Newsletter', 'predictions': "['Newsletter']"},
 {'ground_truth': 'Textbooks', 'predictions': "['Books']"},
 {'ground_truth': 'Disposable spoon', 'predictions': "['Disposable spoon']"},
 {'ground_truth': 'Computer mouse', 'predictions': "['Computer mouse']"},
 {'ground_truth': 'bodywash bottle',
  'predictions': "['Soap bottle', 'Shampoo bottle', 'Body lotion bottle']"}]

In [38]:
len(result_list)

154

In [39]:
result_list[0]

{'ground_truth': 'Newsletter', 'predictions': "['Newsletter']"}

Use chat model or LLM to evaluate the predictions accuracy

In [40]:
# Define the json output
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List

class Similarity(BaseModel):
    similarity: float = Field(description="List of similarity score of the pair of prediction and the corresponding ground truth")


In [41]:
from langchain_openai import ChatOpenAI

qa_model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
qa_model_json_output = qa_model.with_structured_output(
    Similarity, method="json_mode"
)


In [42]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = """ 
You are given a JSON input with the following schema:
{{
    "ground_truth": str,
    "prediction": List[str]
}}

Please evaluate the similarity of the ground truth and the model's prediction and return the similarity as a float between 0 and 1.0. 

Return the result as a JSON output according to the following schema:
{{
    "similarity": float
}}

Examples of how to calculate similarity:
Example 1
Given:
{{
    "ground_truth": "Shampoo bottle",
    "prediction": ["Shampoo bottle", "Conditioner Bottle"]
}}

Expected output:
{{
    "similarity": 1.0
}}

Example 2
Given:
{{
    "ground_truth": "Paper cup",
    "prediction": ["Paper cup"]
}}

Expected output:
{{
    "similarity": 1.0
}}

Example 3
Given:
{{
    "ground_truth": "Paper cup",
    "prediction": ["Plastic cup"]
}}

Expected output:
{{
    "similarity": 0.0
}}

Example 4
Given:
{{
    "ground_truth": "Milk bottle",
    "prediction": ["Bottled milk"]
}}

Expected output:
{{
    "similarity": 1.0
}}

Example 5
Given:
{{
    "ground_truth": "Plastic packaging with bubble wrap",
    "prediction": ["Bubble wrap"]
}}

Expected output:
{{
    "similarity": 0.5
}}


Here is the JSON input: {ground_truth_prediction_json}
"""

prompt_template = ChatPromptTemplate.from_template(prompt)

chain = prompt_template | qa_model_json_output

# @retry_after_delay
# def batch_evaluate(json_input):
#     eval_res = chain.invoke({"json_input": json_input})
#     return eval_res
# eval_res = chain.invoke({"json_input": result_list})
# eval_res


In [43]:
ground_truth_prediction_json_list = [{"ground_truth": ground_truth, "prediction": prediction} for ground_truth, prediction in zip(df['ground_truth'], df['predictions'])]
ground_truth_prediction_json_list[:5]



[{'ground_truth': 'Newsletter', 'prediction': "['Newsletter']"},
 {'ground_truth': 'Textbooks', 'prediction': "['Books']"},
 {'ground_truth': 'Disposable spoon', 'prediction': "['Disposable spoon']"},
 {'ground_truth': 'Computer mouse', 'prediction': "['Computer mouse']"},
 {'ground_truth': 'bodywash bottle',
  'prediction': "['Soap bottle', 'Shampoo bottle', 'Body lotion bottle']"}]

In [44]:
res = chain.batch([{'ground_truth_prediction_json': ground_truth_prediction_json} for ground_truth_prediction_json in ground_truth_prediction_json_list[:5]])

res

[Similarity(similarity=1.0),
 Similarity(similarity=0.0),
 Similarity(similarity=1.0),
 Similarity(similarity=1.0),
 Similarity(similarity=0.0)]

In [45]:
@retry_after_delay
def batch_invoke_evaluate(ground_truth_prediction_json_list, **kwargs):
    batch_res = chain.batch([{'ground_truth_prediction_json': ground_truth_prediction_json} for ground_truth_prediction_json in ground_truth_prediction_json_list])

    return batch_res

In [46]:
# Checkpoint save to csv
import csv
def save_to_csv(csv_filename: str, ground_truth_prediction_json_list: List[str], res: List[Similarity]):
    file_exists = os.path.exists(csv_filename)
    new_rows = [(ground_truth_prediction_json, res.similarity) for ground_truth_prediction_json, res in zip(ground_truth_prediction_json_list, res)]
    with open(csv_filename, 'a', newline='') as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(['ground_truth_prediction_json', 'similarity'])
        writer.writerows(new_rows)

    print(f'Saved {len(ground_truth_prediction_json_list)} rows to {csv_filename}')


In [47]:
# Batch invoke the model, retrying after 65 seconds on rate limit error
import tiktoken
encoding = tiktoken.encoding_name_for_model('gpt-3.5-turbo')

start_index = 0
end_index = 0
curr_tokens = 0

# obtained from openai project limits based on project tier
TPM = 60_000

res = []

results_csv_filename = 'evaluation_results_batch3_with_similar_items.csv'

while end_index < len(ground_truth_prediction_json_list):
    ground_truth_prediction_json = ground_truth_prediction_json_list[end_index]
    prompt_tokens_count = num_tokens_from_string(prompt_template.format(ground_truth_prediction_json=ground_truth_prediction_json), encoding)

    curr_tokens += prompt_tokens_count + 1024 # last constant at the end is buffer for input and output formatting + the response from the model

    if curr_tokens < TPM:
        end_index += 1

    else:
        print(f'Exceeding {TPM} TPM. Current tokens {curr_tokens}. Start index {start_index}. End index {end_index}')
        batch_to_invoke = ground_truth_prediction_json_list[start_index: end_index]

        batch_res = batch_invoke_evaluate(batch_to_invoke) # batch_res is List[Items]
        save_to_csv(results_csv_filename, batch_to_invoke, batch_res)

        start_index = end_index
        res.extend(batch_res)

        curr_tokens = 0 # Rest token count for next batch

    if end_index == len(ground_truth_prediction_json_list):
        print('Last file reached.')
        batch_to_invoke = ground_truth_prediction_json_list[start_index: end_index]
        batch_res = batch_invoke_evaluate(batch_to_invoke) # batch_res is List[Items]
        save_to_csv(results_csv_filename, batch_to_invoke, batch_res)
        res.extend(batch_res)

res



Exceeding 60000 TPM. Current tokens 60871. Start index 0. End index 44
Saved 44 rows to evaluation_results_batch3_with_similar_items.csv
Exceeding 60000 TPM. Current tokens 60877. Start index 44. End index 88
Saved 44 rows to evaluation_results_batch3_with_similar_items.csv
Exceeding 60000 TPM. Current tokens 60919. Start index 88. End index 132
Saved 44 rows to evaluation_results_batch3_with_similar_items.csv
Last file reached.
Saved 22 rows to evaluation_results_batch3_with_similar_items.csv


[Similarity(similarity=1.0),
 Similarity(similarity=0.0),
 Similarity(similarity=1.0),
 Similarity(similarity=1.0),
 Similarity(similarity=0.0),
 Similarity(similarity=0.0),
 Similarity(similarity=1.0),
 Similarity(similarity=0.0),
 Similarity(similarity=1.0),
 Similarity(similarity=0.0),
 Similarity(similarity=1.0),
 Similarity(similarity=1.0),
 Similarity(similarity=1.0),
 Similarity(similarity=1.0),
 Similarity(similarity=0.6666666666666666),
 Similarity(similarity=0.0),
 Similarity(similarity=1.0),
 Similarity(similarity=1.0),
 Similarity(similarity=1.0),
 Similarity(similarity=1.0),
 Similarity(similarity=0.0),
 Similarity(similarity=0.0),
 Similarity(similarity=1.0),
 Similarity(similarity=1.0),
 Similarity(similarity=1.0),
 Similarity(similarity=1.0),
 Similarity(similarity=1.0),
 Similarity(similarity=0.5),
 Similarity(similarity=1.0),
 Similarity(similarity=1.0),
 Similarity(similarity=1.0),
 Similarity(similarity=0.0),
 Similarity(similarity=0.0),
 Similarity(similarity=0.0),

In [48]:
len(res)

154

put similarity into dataframe

In [49]:
df['similarity'] = [r.similarity for r in res]
df.head()


,filename,predictions,ground_truth,similarity
0,../images/test/batch3/Newsletter (4).jpeg,['Newsletter'],Newsletter,1.0
1,../images/test/batch3/Textbooks (3).jpg,['Books'],Textbooks,0.0
2,../images/test/batch3/Disposable spoon (3).jpg,['Disposable spoon'],Disposable spoon,1.0
3,../images/test/batch3/Computer mouse (2).jpg,['Computer mouse'],Computer mouse,1.0
4,../images/test/batch3/bodywash bottle (3).jpeg,"['Soap bottle', 'Shampoo bottle', 'Body lotion...",bodywash bottle,0.0


Output to csv for manual evaluation

In [50]:
df[['filename', 'ground_truth', 'predictions', 'similarity']].to_csv('manual_evaluation_batch3_with_similar_items.csv', index=False)



# Put similar items into new entries in the json file

In [33]:
import json
with open('data.json', 'r') as file:
    data = json.load(file)

len(data)



322

In [34]:
unique_items = set()
res = []
for item in data:
    item_name = item['item']
    item_name = item_name.strip()
    item_name = item_name.capitalize()
    if item_name not in unique_items:
        res.append({
        'material': item['material'],
        'item': item['item'],
        'recyclable': item['recyclable'],
        'instructions': item['instructions']
        })
        unique_items.add(item_name)

    for similar_item in item['similar_items']:
        similar_item = similar_item.strip()
        similar_item = similar_item.capitalize()
        if similar_item not in unique_items:
            res.append({
                'material': item['material'],
                'item': similar_item,
                'recyclable': item['recyclable'],
                'instructions': item['instructions']
            })
            unique_items.add(similar_item)


len(res)




435

In [35]:
sorted_res = sorted(res, key=lambda item: item['item'])




In [36]:
with open('cleaned_data.json', 'w') as file:
    json.dump(sorted_res, file, indent=4)

